In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
import time

# 设置平台  GPU/ CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
num_workers=16

# 定义Swish激活函数
class Swish(nn.Module):
    def __init__(self, beta=1.0):
        super(Swish, self).__init__()
        self.beta = beta

    def forward(self, x):
        return x * torch.sigmoid(self.beta * x)
start_time = time.time()
# 定义前馈神经网络模型
class FeedforwardNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(FeedforwardNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        #self.relu = nn.LeakyReLU(negative_slope=0.01)
        self.swish = Swish()  # 使用Swish激活函数
        self.fc2 = nn.Linear(hidden_size, 128)
        #self.relu2 = nn.LeakyReLU(negative_slope=0.01)
        self.swish2 = Swish()  # 使用Swish激活函数
        self.fc3 = nn.Linear(128, num_classes)
        
    def forward(self, x):
        out = self.fc1(x)
        out = self.swish(out)
        out = self.fc2(out)
        out = self.swish2(out)
        out = self.fc3(out)
        return out

# 定义超参数
input_size = 28 * 28  # 输入特征的维度
hidden_size = 512  # 隐藏层的大小
num_classes = 10  # 类别的数量
learning_rate = 0.001  # 学习率
num_epochs = 20  # 迭代次数
batch_size = 64  # 批次大小

# 加载数据集
train_dataset = MNIST(root='~/Datasets/MNIST', train=True, transform=ToTensor(), download=True)
test_dataset = MNIST(root='~/Datasets/MNIST', train=False, transform=ToTensor(), download=False)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True,num_workers=num_workers)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False,num_workers=num_workers)

# 初始化模型
model = FeedforwardNN(input_size, hidden_size, num_classes).to(device)

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


#画图
losses = []
accuracy = []
# 训练模型
for epoch in range(num_epochs):
    model.train()
    for batch_idx, (data, targets) in enumerate(train_loader):
        data = data.view(-1, input_size).to(device)
        targets = targets.to(device)
        # 前向传播
        outputs = model(data)
        loss = criterion(outputs, targets)
        
        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (batch_idx+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{batch_idx+1}/{len(train_loader)}], Loss: {loss.item():.4f}')
    _, predicted = torch.max(outputs.data, 1)
    acc = (predicted == targets).sum().item() / targets.size(0)
    accuracy.append(acc)
    # 记录损失和准确率
    losses.append(loss.item())
    print(f'Train Accuracy: {acc:.4f}')
    
    
    # 在测试集上评估模型
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for data, targets in test_loader:
            data = data.view(-1, input_size).to(device)
            targets = targets.to(device)
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
        
        
        acc = correct / total
        print(f'Test Accuracy: {acc:.4f}')
        
            
end_time = time.time()
training_time = end_time - start_time
print("模型的training_time为"+str(round(training_time,2))+"秒" )

cuda
Epoch [1/20], Step [100/938], Loss: 0.2417
Epoch [1/20], Step [200/938], Loss: 0.3634
Epoch [1/20], Step [300/938], Loss: 0.2861
Epoch [1/20], Step [400/938], Loss: 0.3255
Epoch [1/20], Step [500/938], Loss: 0.1095
Epoch [1/20], Step [600/938], Loss: 0.1077
Epoch [1/20], Step [700/938], Loss: 0.0545
Epoch [1/20], Step [800/938], Loss: 0.1009
Epoch [1/20], Step [900/938], Loss: 0.0891
Train Accuracy: 0.9688
Test Accuracy: 0.9601
Epoch [2/20], Step [100/938], Loss: 0.0545
Epoch [2/20], Step [200/938], Loss: 0.1631
Epoch [2/20], Step [300/938], Loss: 0.1590
Epoch [2/20], Step [400/938], Loss: 0.0787
Epoch [2/20], Step [500/938], Loss: 0.1118
Epoch [2/20], Step [600/938], Loss: 0.0403
Epoch [2/20], Step [700/938], Loss: 0.0349
Epoch [2/20], Step [800/938], Loss: 0.0421
Epoch [2/20], Step [900/938], Loss: 0.0342
Train Accuracy: 1.0000
Test Accuracy: 0.9695
Epoch [3/20], Step [100/938], Loss: 0.0120
Epoch [3/20], Step [200/938], Loss: 0.1014
Epoch [3/20], Step [300/938], Loss: 0.0791
Ep

Epoch [19/20], Step [900/938], Loss: 0.0004
Train Accuracy: 1.0000
Test Accuracy: 0.9820
Epoch [20/20], Step [100/938], Loss: 0.0013
Epoch [20/20], Step [200/938], Loss: 0.0006
Epoch [20/20], Step [300/938], Loss: 0.0000
Epoch [20/20], Step [400/938], Loss: 0.0000
Epoch [20/20], Step [500/938], Loss: 0.0007
Epoch [20/20], Step [600/938], Loss: 0.0004
Epoch [20/20], Step [700/938], Loss: 0.0004
Epoch [20/20], Step [800/938], Loss: 0.0000
Epoch [20/20], Step [900/938], Loss: 0.0004
Train Accuracy: 1.0000
Test Accuracy: 0.9835
模型的training_time为754.27秒
